# Spotify CSV

In [46]:
## Import libraries
import requests
import pandas as pd
import pymysql
from sqlalchemy import create_engine


In [47]:
## Connect to MySQL DB
##@TODO: change credentials to final DB
    
driver   = 'mysql+pymysql:'
user     = 'root' ## the user you use to access the DB
password = 'ML_LI_04' ## the password you use to access the DB
ip       = '35.195.172.42' ## the DB host IP
database = 'data_thieves' ## name_of_DB_you_want_to_connect

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

### Import raw CSV

In [48]:
### https://www.kaggle.com/zaheenhamidani/ultimate-spotify-tracks-db
raw_data = pd.read_csv('ultimate_spotify_tracks_db/SpotifyFeatures.csv')
original = raw_data.copy()

In [49]:
raw_data.head(3)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.0,C#,0.346,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.0,F#,0.151,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.0,C,0.103,-13.879,Minor,0.0362,99.488,5/4,0.368


In [50]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232725 entries, 0 to 232724
Data columns (total 18 columns):
genre               232725 non-null object
artist_name         232725 non-null object
track_name          232725 non-null object
track_id            232725 non-null object
popularity          232725 non-null int64
acousticness        232725 non-null float64
danceability        232725 non-null float64
duration_ms         232725 non-null int64
energy              232725 non-null float64
instrumentalness    232725 non-null float64
key                 232725 non-null object
liveness            232725 non-null float64
loudness            232725 non-null float64
mode                232725 non-null object
speechiness         232725 non-null float64
tempo               232725 non-null float64
time_signature      232725 non-null object
valence             232725 non-null float64
dtypes: float64(9), int64(2), object(7)
memory usage: 32.0+ MB


In [51]:
raw_data.columns

Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

### Normalize raw CSV

In [52]:
## Check if each track_name is unique
print('Raw data DF length: ', len(raw_data))

unique_tracks = raw_data.track_name.unique()
print('Unique track name values: ', len(unique_tracks))

Raw data DF length:  232725
Unique track name values:  148615


In [53]:
## HOW DO WE WANT TO PROCEED??

In [54]:
## Unify values 'Children's Music' and 'Children’s Music' in genre column

raw_data['genre'].replace('Children’s Music', 'Children\'s Music', inplace = True)
print('Raw data DF length after genre replace: ', len(raw_data))


Raw data DF length after genre replace:  232725



### Create artists table

In [55]:
## Extract series 'artist_name' from the raw data
artists = raw_data['artist_name'].copy()
print('Len artist name: ', len(artists))

Len artist name:  232725


In [56]:
## Delete duplicated values
artists.drop_duplicates(keep='first', inplace=True)
print('Len artist name unique: ', len(artists))

Len artist name unique:  14564


In [57]:
artists

0                   Henri Salvador
1                Martin & les fées
2                  Joseph Williams
4                     Fabien Nataf
7                      Laura Mayne
                    ...           
232536                    Yahzarah
232557                Amber-Simone
232619                  Dharmasoul
232637                        Swim
232721    Jr Thomas & The Volcanos
Name: artist_name, Length: 14564, dtype: object

In [58]:
## Export it to MySQL
artists.to_sql('artist', con=engine, if_exists = 'append', index = False) 

In [59]:
##Import it back with the table PK
artists_with_id = pd.read_sql('artist', engine)
print('Len artists after DB: ', len(artists_with_id))

Len artists after DB:  14564


In [60]:
artists_with_id.head(3)

,ID,artist_name
0,1,Henri Salvador
1,2,Martin & les fées
2,3,Joseph Williams


In [61]:
##Add the artist_id to the raw table
raw_with_artist_id = pd.merge(raw_data, artists_with_id, on='artist_name')

In [62]:
##Change name of artist_id column
raw_with_artist_id = raw_with_artist_id.rename(columns={'ID':'artist_id'})

In [63]:
print('Raw data DF length after adding artist ID: ', len(raw_with_artist_id))

Raw data DF length after adding artist ID:  232725



### Create genre table

In [64]:
## Extract series 'genre' from the raw data
genres = raw_data['genre'].copy()
print('Len genres: ', len(genres))

Len genres:  232725


In [65]:
## Delete duplicated values
genres.drop_duplicates(keep='first', inplace=True)
print('Len genres unique: ', len(genres))

Len genres unique:  26


In [66]:
genres.name = 'genre_name'

In [67]:
## Export it to MySQL
genres.to_sql('genre', con=engine, if_exists = 'append', index = False) 


In [68]:
##Import it back with the table PK
genres_with_id = pd.read_sql('genre', engine)
print('Len genres after DB: ', len(genres_with_id))

Len genres after DB:  26


In [69]:
genres_with_id = genres_with_id.rename(columns={'genre_name':'genre'})
genres_with_id.head(3)

,ID,genre
0,1,Movie
1,2,R&B
2,3,A Capella


In [70]:
##Add the genre_id to the raw table
raw_with_artist_genre_id = pd.merge(raw_with_artist_id, genres_with_id, on='genre')

In [71]:
##Change name of artist_id column
raw_with_artist_genre_id = raw_with_artist_genre_id.rename(columns={'ID':'genre_id'})
raw_with_artist_genre_id.head(3)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_id,genre_id
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.9100,0.0,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814,1,1
1,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.3260,0.0,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227,1,1
2,Movie,Henri Salvador,Le petit souper aux chandelles,0Mf1jKa8eNAf1a4PwTbizj,0,0.749,0.578,160627,0.0948,0.0,C#,0.1070,-14.970,Major,0.1430,87.479,4/4,0.358,1,1


In [72]:
print('Raw data DF length after adding genre ID: ', len(raw_with_artist_genre_id))

Raw data DF length after adding genre ID:  232725



### Create track table

Track table should include all the information on the tracks + the artist ID as a FK

In [73]:
## Check all the columns available
raw_with_artist_genre_id.columns

Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'artist_id',
       'genre_id'],
      dtype='object')

In [74]:
## Create a table only with the columns that belong to the track table
tracks = raw_with_artist_genre_id[['track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'artist_id']]

In [75]:
##Change the names to meet the DB columns
tracks.columns = ['track_name', 'spotify_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'track_key', 'liveness', 'loudness', 'track_mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'artist_id']

In [76]:
tracks.head(3)

,track_name,spotify_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,track_key,liveness,loudness,track_mode,speechiness,tempo,time_signature,valence,artist_id
0,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.9100,0.0,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814,1
1,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.3260,0.0,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227,1
2,Le petit souper aux chandelles,0Mf1jKa8eNAf1a4PwTbizj,0,0.749,0.578,160627,0.0948,0.0,C#,0.1070,-14.970,Major,0.1430,87.479,4/4,0.358,1


In [77]:
print('Len tracks: ', len(tracks))

Len tracks:  232725


In [78]:
## Export it to MySQL
tracks.to_sql('track', con=engine, if_exists = 'append', index = False)

In [79]:
##Import it back with the table PK
tracks_with_id = pd.read_sql('track', engine)
print('Len tracks after DB: ', len(tracks_with_id))

Len tracks after DB:  232725


### Create track_genre relationship table

This table should include the track_id and the genre_id

In [80]:
## Create a subset of the tracks_with_id table to merge
tracks_with_id_to_merge = tracks_with_id[['ID', 'spotify_id']]

# Change column names to meet the DB table names
tracks_with_id_to_merge.columns = ['track_id', 'spotify_id']

In [81]:
## Create a subset of the raw_with_artist_genre_id table to merge
raw_with_artist_genre_id_to_merge = raw_with_artist_genre_id[['track_id', 'genre_id']]

# Change column names to meet the DB table names
raw_with_artist_genre_id_to_merge.columns = ['spotify_id', 'genre_id']

In [82]:
##Merge tables
track_genre = pd.merge(tracks_with_id_to_merge, raw_with_artist_genre_id_to_merge, on='spotify_id')

##Drop spotify_id column
track_genre.drop(columns=['spotify_id'], inplace = True)

In [83]:
## Export it to MySQL
track_genre.to_sql('track_genre', con=engine, if_exists = 'append', index = False) 

In [84]:
##Import it back with the table PK
track_genre_with_id = pd.read_sql('track', engine)
print('Len track-genre table after DB: ', len(track_genre_with_id))

Len track-genre table after DB:  232725
